<a href="https://colab.research.google.com/github/janbanot/msc-cs-code/blob/main/sem3/DL/DL_2025_Lab6-c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install torchinfo

# Import potrzebnych modułów i funkcji
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from torchinfo import summary

In [ ]:
%%html
<!-- Potrzebne dla poprawnego wyświetlania paska postępu tqdm w VSCode https://stackoverflow.com/a/77566731 -->
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}
</style>

# Modele generatywne -- część II

## Flow Matching & Stochastic Flow Matching

### Przykład 1

Poniższy przykład bazuje na [przykładzie](https://github.com/facebookresearch/flow_matching/blob/main/examples/standalone_flow_matching.ipynb) z biblioteki `flow_matching`

*Celem* jest trening modelu, który umożliwia przekształcenie próbek z rozkładu początkowego
w próbki z rozkładu docelowego (danych). Rozkład początkowy to _rozkład normalny_ (Gaussa),
natomiast końcowy to rozkład _dwóch półksiężyców_ próbkowany za pomocą `sklearn.datasets.make_moons`

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons

# Generate 200 samples using make_moons
X, _ = make_moons(n_samples=200, noise=0.1, random_state=41)

# Create the plot
plt.figure(figsize=(6, 4))
plt.scatter(X[:, 0], X[:, 1])
plt.title("2D samples from sklearn.datasets.make_moons")
plt.xlabel("X1")
plt.ylabel("X2")
plt.grid(True)
plt.axis('equal')
plt.show()


#### Model

Model przybliża *pole wektorowe*, które dla podanego pkt. oraz chwili czasu $t \in [0, 1]$ wskazuje
wektor prędkości wg, którego powinien być przesunięty.

In [ ]:
import torch
import torch.optim as optim
from torch import nn, Tensor


class Flow(nn.Module):
    " Perceptron z 2 warstwami ukrytymi "

    def __init__(self, input_dim=2, hidden_dim=64):
        super(Flow, self).__init__()
        time_dim = 1
        self.encoder = nn.Sequential(
            nn.Linear(input_dim + time_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x, t):
        """
        Wejściem sieci jest aktualne położenie pkt. oraz czas t.
        Wynikiem jest wektor prędkości (2 wymiarowy).
        """
        return self.encoder(torch.cat([x, t], dim=1))

### Krok treningowy

In [ ]:
def train_step(model, optimizer, x_1, stochastic=False):
    """
    Wykonuje jeden krok treningowy dla modelu Flow Matching.

    Kroki:
      1. Próbkuj szum (x_0) z rozkładu normalnego.
      2. Wylosuj czas t dla każdego przykładu docelowego x_1.
      3. Wygeneruj zaszumiony obraz (x_t) jako interpolację między szumem a "czystym" przykładem.
      4. Zdefiniuj cel jako różnicę (prędkość) między czystym przykładem a szumem.
      5. Naucz model przewidywać tę prędkość.

    Zwraca:
      float: Obliczona wartość funkcji straty.
    """
    model.train()
    batch_size = x_1.size(0)

    # 1. Próbkowanie szumu z rozkładu normalnego.
    x_0 = torch.randn_like(x_1, device=x_1.device)

    # 2. Próbkowanie czasu t z rozkładu jednostajnego dla każdego przykładu, kształt [B, 1, ...].
    t = torch.rand([batch_size] + [1]*(x_0.ndim - 1), device=x_1.device)

    # 3. Obliczenie zaszumionego przykładu x_t jako interpolacji między x_0 a x_1.
    # Zakładamy, że punkty poruszają się po prostej łączącej x_0 z x_1 (ścieżka liniowa)
    x_t = (1 - t) * x_0 + t * x_1

    if stochastic:  # Przypadek stochastyczny (SFM): Dodaj szum zgodnie z harmonogramem sigma_t
        sigma_t = calculate_sigma(t)
        noise = torch.randn_like(x_1) # Próbkowanie szumu z rozkładu normalnego
        x_t = x_t + sigma_t * noise

    # 4. Obliczenie docelowej prędkości -- dla ścieżki liniowej
    target_velocity = x_1 - x_0

    # 5. Predykcja prędkości za pomocą modelu
    predicted_velocity = model(x_t, t.view(-1, 1))

    loss = torch.mean((predicted_velocity - target_velocity)**2)  # MSE

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()


def calculate_sigma(t):
    """
    Funkcja definiująca harmonogram szumu dla SFM.
    Prosty przebieg sinusoidalny – maksimum przy t=0.5, zero przy t=0 i t=1.
    """
    return 0.01 * torch.sin(torch.pi * t) + 1e-4 # Dodaj małe epsilon dla stabilności

### Trening

In [ ]:
model = Flow()
optimizer = optim.AdamW(model.parameters(), lr=0.005)

num_epochs = 20
for epoch in range(num_epochs):
    total_loss = 0
    num_batches = 1000
    for _ in range(num_batches):
        batch_size = 256
        x1 = Tensor(make_moons(batch_size, noise=0.05)[0])
        loss_val = train_step(model, optimizer, x1, stochastic=False)
        total_loss += loss_val

    avg_loss = total_loss / num_batches
    print(f'[{epoch}] {avg_loss = :.3f}')

### Próbkowanie

In [ ]:
@torch.no_grad()
def sample_trajectory(model, shape, device='cpu', steps=100, samples=1, checkpoint_every=10):
    """
    Generuje trajektorie próbek z modelu flow-matching metodą Eulera.

    Rozpoczyna przy t=0 od szumu gaussowskiego, a następnie iteracyjnie aktualizuje:
        x ← x + v(x, t) * Δt
    aż do t=1, zapisując stan co checkpoint_every kroków.

    Args:
        model:       wytrenowany model predykcji prędkości v(x, t).
        shape:       kształt jednej próbki (np. (1, 28, 28)).
        steps:       liczba kroków Eulera od t=0 do t=1.
        samples:     liczba próbek w batchu.
        device:      'cpu' lub 'cuda'.
        checkpoint_every: odstęp kroków między zapisem stanu.

    Returns:
        Tensor o kształcie [num_checkpoints, samples, *shape]
        zawierający zapisane stany x (przeniesione na CPU).
    """
    device = torch.device(device)
    model.eval()
    # Inicjalizacja od losowego szumu dla t=0
    x = torch.randn(samples, *shape, device=device)

    time_steps = torch.linspace(0, 1, steps + 1, device=device)
    delta_t = time_steps[1] - time_steps[0]

    # Ile stanów będziemy zapisywać
    num_ckpt = steps // checkpoint_every
    ckpt_idx = 0
    traj = torch.empty(num_ckpt, samples, *shape, device=device)
    for i in range(steps):
        # Tworzymy wektor czasu o rozmiarze batcha
        t = time_steps[i].repeat(samples, 1)

        v = model(x, t)   # Predykcja prędkości v(x, t)

        # Aktualizacja Eulera w miejscu: x ← x + v * Δt
        x = x + v * delta_t

        if (i+1) % checkpoint_every == 0:
            traj[ckpt_idx] = x
            ckpt_idx += 1

    return traj.reshape(-1, samples, *shape).cpu()


@torch.no_grad()
def sample(model, num_samples, data_shape, device, num_steps=100):
    " Próbkowanie z modelu FM/SFM (ostatni pkt. trajektorii) "

    model.eval()
    device = torch.device(device)
    # Inicjalizacja: szum w chwili t=0
    x = torch.randn(num_samples, *data_shape, device=device)

    time_steps = torch.linspace(0, 1, num_steps + 1, device=device)
    delta_t = time_steps[1] - time_steps[0]

    for i in range(num_steps):
        t = time_steps[i].repeat(num_samples, 1)
        v = model(x, t)
        x = x + v * delta_t  # Krok Eulera
    return x.cpu()

In [ ]:
dim = 2
trajectory = sample_trajectory(model, (dim, ), samples=200)
n_steps = trajectory.shape[0]

fig, axes = plt.subplots(1, n_steps + 1, figsize=(30, 4), sharex=True, sharey=True)
time_steps = torch.linspace(0, 1.0, n_steps + 1)

x = trajectory[0].detach()
axes[0].scatter(x[:, 0], x[:, 1], s=10)
axes[0].set_title(f't = {time_steps[0]:.2f}')
axes[0].set_xlim(-3.0, 3.0)
axes[0].set_ylim(-3.0, 3.0)

for i in range(n_steps):
    x = trajectory[i].detach()
    axes[i + 1].scatter(x[:, 0], x[:, 1], s=10)
    axes[i + 1].set_title(f't = {time_steps[i + 1]:.2f}')

plt.tight_layout()
plt.show()

### Przypadek stochastyczny

In [ ]:
model = Flow()
optimizer = optim.AdamW(model.parameters(), lr=0.005)

num_epochs = 20
for epoch in range(num_epochs):
    total_loss = 0
    num_batches = 1000
    for _ in range(num_batches):
        batch_size = 256
        x1 = Tensor(make_moons(batch_size, noise=0.05)[0])
        loss_val = train_step(model, optimizer, x1, stochastic=True)
        total_loss += loss_val

    avg_loss = total_loss / num_batches
    print(f'[{epoch}] {avg_loss = :.3f}')

In [ ]:
dim = 2
trajectory = sample_trajectory(model, (dim, ), samples=200)
n_steps = trajectory.shape[0]

fig, axes = plt.subplots(1, n_steps + 1, figsize=(30, 4), sharex=True, sharey=True)
time_steps = torch.linspace(0, 1.0, n_steps + 1)

x = trajectory[0].detach()
axes[0].scatter(x[:, 0], x[:, 1], s=10)
axes[0].set_title(f't = {time_steps[0]:.2f}')
axes[0].set_xlim(-3.0, 3.0)
axes[0].set_ylim(-3.0, 3.0)

for i in range(n_steps):
    x = trajectory[i].detach()
    axes[i + 1].scatter(x[:, 0], x[:, 1], s=10)
    axes[i + 1].set_title(f't = {time_steps[i + 1]:.2f}')

plt.tight_layout()
plt.show()

## Przykład 2. MNIST

In [ ]:
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Ogranicz do wybranych cyfr
def filter_indices(dataset, target_digits=[4, 7]):
    indices = []
    for idx in range(len(dataset)):
        if dataset[idx][1] in target_digits:
            indices.append(idx)
    return indices

# Indeksy dla obrazów wybranych cyfr
train_indices = filter_indices(train_dataset)
test_indices = filter_indices(test_dataset)

# Podzbiory
train_subset = Subset(train_dataset, train_indices)
test_subset = Subset(test_dataset, test_indices)

batch_size = 64
train_loader = DataLoader(dataset=train_subset, batch_size=batch_size, shuffle=True,
                          pin_memory=True, num_workers=2, persistent_workers=True)
test_loader = DataLoader(dataset=test_subset, batch_size=batch_size, shuffle=False)

### Model

Spróbujemy z prostą siecią typu MLP

In [ ]:
def get_device():   # Obliczenia wykonamy na GPU, jeżeli jest dostępne, a na CPU w przeciwny razie
    return torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class TimeEmbeddingNet(nn.Module):
    """
    Sieć do kodowania czasu do wektora o wymiarach time_emb_dim
    Wej: t, shape [B, 1]
    Wyj: wektor czasu [B, time_emb_dim]
    """
    def __init__(self, time_emb_dim=128):
        super(TimeEmbeddingNet, self).__init__()
        self.linear1 = nn.Linear(1, time_emb_dim)
        self.relu = nn.LeakyReLU()
        self.linear2 = nn.Linear(time_emb_dim, time_emb_dim)

    def forward(self, t):   # t: [B, 1]
        t_emb = self.relu(self.linear1(t))
        t_emb = self.relu(self.linear2(t_emb))
        return t_emb  # [B, time_emb_dim]


class FlowMNIST(nn.Module):
    def __init__(self, input_dim, hidden_dim=1024, time_dim=128):
        super(FlowMNIST, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(input_dim + time_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, input_dim)
        )

        self.time_emb = TimeEmbeddingNet(time_emb_dim=time_dim)

    def forward(self, x, t):
        t = self.time_emb(t)
        return self.encoder(torch.cat([x, t], dim=1))

In [ ]:
from torch.optim.lr_scheduler import MultiStepLR


model = FlowMNIST(28 * 28, time_dim=16)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
mse_loss = nn.MSELoss()

model = model.to(get_device())
num_epochs = 7
scheduler = MultiStepLR(optimizer, milestones=[4,6], gamma=0.1)

for epoch in range(num_epochs):
    total_loss = 0
    for batch_x1, _ in tqdm(train_loader):
        x1 = batch_x1.view(-1, 28*28)
        loss_val = train_step(model, optimizer, x1.to(get_device()), stochastic=False)
        total_loss += loss_val
    scheduler.step()
    avg_loss = total_loss / len(train_loader)
    print(f'{avg_loss = :.3f}')

### Trajektoria i próbkowanie

In [ ]:
import torchvision.utils as vutils


def show_images(images):
    images = torch.stack([(img - img.min()) / (img.max() - img.min() + 1e-5) for img in images])
    nrow = ncol = int(len(images) ** 0.5 + 0.5)
    grid = vutils.make_grid(images, nrow=nrow, padding=2)
    np_grid = grid.numpy().transpose((1, 2, 0))
    plt.figure(figsize=(ncol, nrow))
    plt.imshow(np_grid, cmap='gray')
    plt.axis('off')


show_images( sample_trajectory(model, (28 * 28, ), device=get_device()).view(-1, 1, 28, 28))

In [ ]:
samples = sample(model, 9, (28 * 28, ), get_device())
show_images(samples.view(9, 1, 28, 28))

### U-Net

Uzyskanie wyników lepszej jakości wymaga modelu o architekturze lepiej dopasowanej do zadania, np. sieci U-Net

In [ ]:
class ResidualBlock(nn.Module):
    """
    Podstawowy blok rezydualny z warunkowaniem czasowym wykonujący:
      - Dwie warstwy konwolucyjne.
      - Warunkowanie czasowe przez wyuczoną projekcję liniową wspólnego osadzenia czasowego.
      - Dopasowanie połączenia rezydualnego, jeśli liczba kanałów wejścia i wyjścia się różni.
    """
    def __init__(self, in_channels, out_channels, time_emb_dim):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, groups=out_channels // 16)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, groups=out_channels // 32)
        # Projekcja wspólnego osadzenia czasowego na wektor cech do transmisji.
        self.time_mlp = nn.Linear(time_emb_dim, out_channels)
        self.relu = nn.LeakyReLU(inplace=True)
        # Dopasowanie połączenia rezydualnego, jeśli wymiary kanałów są różne.
        self.res_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1) if in_channels != out_channels else None
        self.norm = nn.GroupNorm(num_groups=in_channels // 8, num_channels=in_channels)

    def forward(self, x, t_emb):
        h = self.relu(self.conv1(self.norm(x)))
        # Projekcja osadzenia czasowego i dodanie go do mapy cech (broadcasting po wymiarach przestrzennych).
        time_feature = self.time_mlp(t_emb).unsqueeze(-1).unsqueeze(-1)
        h = h + time_feature
        h = self.relu(self.conv2(h))
        # Zastosowanie połączenia rezydualnego.
        residual = self.res_conv(x) if self.res_conv is not None else x
        return h + residual


class DownBlock(nn.Module):
    """
    Blok zmniejszający rozdzielczość (koder), stosujący blok rezydualny, a następnie konwolucję ze skokiem.
    Zwraca zarówno cechy z bloku rezydualnego (do połączenia typu skip), jak i wyjście o zmniejszonej rozdzielczości.
    """
    def __init__(self, in_channels, out_channels, time_emb_dim):
        super().__init__()
        self.resblock = ResidualBlock(in_channels, out_channels, time_emb_dim)
        # Downsampling o czynnik 2.
        self.downsample = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=2, padding=1, groups=out_channels//16)

    def forward(self, x, t_emb):
        skip = self.resblock(x, t_emb)
        x_down = self.downsample(skip)
        return skip, x_down


class UpBlock(nn.Module):
    """
    Blok zwiększający rozdzielczość (dekoder), który:
      - Zwiększa rozdzielczość za pomocą transponowanej konwolucji.
      - Konkatenuje wynik upsamplingu z odpowiadającym połączeniem typu skip.
      - Przetwarza połączone cechy przy pomocy bloku rezydualnego.
    """
    def __init__(self, in_channels, skip_channels, out_channels, time_emb_dim):
        super().__init__()
        self.upsample = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1, groups=out_channels//16)
        self.resblock = ResidualBlock(out_channels + skip_channels, out_channels, time_emb_dim)

    def forward(self, x, skip, t_emb):
        x_up = self.upsample(x)
        # Konkatenuj po wymiarze kanałów (dodanie skip jako dodatkowych kanałów).
        x_cat = torch.cat([x_up, skip], dim=1)
        return self.resblock(x_cat, t_emb)


class UNet(nn.Module):
    """
    Model U-Net z warunkowaniem czasowym przystosowany do generowania obrazów w stylu modeli dyfuzyjnych.
    Model:
      1. Oblicza wspólne osadzenie czasowe.
      2. Przetwarza wejściowy obraz za pomocą ścieżki kodującej (zmniejszającej rozdzielczość).
      3. Stosuje blok typu „wąskie gardło”.
      4. Odtwarza wynik przy użyciu ścieżki dekodującej (zwiększającej rozdzielczość) z połączeniami typu skip.
    """
    def __init__(self, time_emb_dim=128, mult=1):
        super().__init__()
        # Wspólna sieć osadzająca czas (zdefiniowana w innym miejscu).
        self.time_embed = TimeEmbeddingNet(time_emb_dim=time_emb_dim)
        self.relu = nn.LeakyReLU(inplace=True)
        # Początkowa konwolucja: zmiana z 1 kanału wejściowego na 32 mapy cech.
        self.initial_conv = nn.Conv2d(1, 32*mult, kernel_size=3, padding=1)

        # Koder / ścieżka zmniejszająca rozdzielczość.
        self.down1 = DownBlock(32*mult, 64*mult, time_emb_dim)
        self.down2 = DownBlock(64*mult, 128*mult, time_emb_dim)

        # Blok wąskiego gardła o najmniejszej rozdzielczości.
        self.bottleneck = ResidualBlock(128*mult, 128*mult, time_emb_dim)

        # Dekoder / ścieżka zwiększająca rozdzielczość.
        self.up1 = UpBlock(128*mult, skip_channels=128*mult, out_channels=64*mult, time_emb_dim=time_emb_dim)
        self.up2 = UpBlock(64*mult, skip_channels=64*mult, out_channels=32*mult, time_emb_dim=time_emb_dim)

        # Ostatnia konwolucja: zmiana na 1 kanał wyjściowy.
        self.final_conv = nn.Conv2d(32*mult, 1, kernel_size=3, padding=1)

    def forward(self, x, t):
        """
        Przepływ danych przez model U-Net.
        x: obrazy wejściowe o kształcie [B, 1, 28, 28].
        t: skalary czasowe o kształcie [B, 1].
        """
        t_emb = self.time_embed(t)
        x = self.relu(self.initial_conv(x))
        skip1, x = self.down1(x, t_emb)
        skip2, x = self.down2(x, t_emb)
        x = self.bottleneck(x, t_emb)
        x = self.up1(x, skip2, t_emb)
        x = self.up2(x, skip1, t_emb)
        return self.final_conv(x)

In [ ]:
model = UNet(time_emb_dim=64).to(get_device())
# display(summary(model))
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = MultiStepLR(optimizer, milestones=[5,6], gamma=0.1)

num_epochs = 7

for epoch in range(num_epochs):
    total_loss = 0.0
    for clean_images, _ in tqdm(train_loader):
        clean_images = clean_images.to(get_device())
        loss_value = train_step(model, optimizer, clean_images, stochastic=True)
        total_loss += loss_value

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}: Loss = {avg_loss:.4f}")
    scheduler.step()

In [ ]:
generated_samples = sample(model, num_samples=16, data_shape=(1, 28, 28), device=get_device())
show_images(generated_samples)

In [ ]:
show_images( sample_trajectory(model, (1, 28, 28), device=get_device()).view(10, 1, 28, 28))

### Alternatywne metody próbkowania

Metoda Eulera rozw. równań różniczkowych (czyli generowania trajektorii od szumu
do danych) jest jedną z prostszych. Potencjalna poprawa jakości generowanych przykładów
jest możliwa, gdy zastosujemy metodę bardziej zaawansowaną, np. dopri5

In [ ]:
!pip install torchdiffeq

In [ ]:
from torchdiffeq import odeint  # Przykładowa biblioteka dla rozw. równań różniczkowych

# model: wytrenowana sieć v_theta(x, t)
# Musi spełniać wymaganą przez solver sygnaturę, np. f(t, x)
class ODEFunc(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, t, x):
        # Solver może przekazać skalarną wartość t, więc trzeba ją rozszerzyć do batcha
        t_vec = torch.ones(x.shape[0], 1, device=x.device) * t
        # Zapewniamy, że t_vec ma format oczekiwany przez model
        # W razie potrzeby dostosować kształt lub wymiary
        return self.model(x, t_vec)


def sample_dopri(model, data_shape, num_samples, device):
    ode_func = ODEFunc(model).to(device)
    x0 = torch.randn(num_samples, *data_shape, device=device)  # Próbka szumu
    time_points = torch.linspace(0, 1, steps=100, device=device)  # Definicja punktów czasowych integracji

    # Rozwiąż równanie różniczkowe: dx/dt = v_theta(x, t)
    # 'odeint' wykonuje numeryczne kroki całkowania
    # rtol, atol sterują dokładnością solvera (dokładność vs szybkość)
    solution = odeint(ode_func, x0, time_points, method='dopri5', rtol=1e-5, atol=1e-5)
    final_samples = solution[-1]  # Ostateczne próbki to stan w ostatnim punkcie czasowym (t=1)
    return final_samples

In [ ]:
samples = sample_dopri(model, (1, 28, 28), num_samples=16, device=get_device())
show_images(samples.cpu())

# DDPM

Odszumiające probabilistyczne modele dyfuzyjne (ang. Denoising Diffusion Probabilistic Models)
są alternatywnym podejściem do generowania. W pewnym sensie są równoważne modelom SFM.

In [ ]:
T = 500
device = get_device()
betas = torch.linspace(1e-4, 1e-2, steps=T).to(device)
alphas = 1 - betas
bar_alphas = torch.cumprod(alphas, dim=-1)
bar_alphas

In [ ]:
# Zaszumianie danych

for batch, _ in train_loader:
    for x0 in batch:
        break

    for t in (0, 10, 100, 200, 300, 400, 499):
        alpha_t = bar_alphas[t]
        noise = torch.randn_like(x0).to(device)
        x_t = alpha_t**0.5 * x0.to(device) + (1-alpha_t)**0.5 * noise
        show_images(x_t.cpu())
    break

In [ ]:
def train_step_ddpm(model, clean_images):
    " Pojedynczy krok treningowy dla DDPM "

    model.train()
    batch_size = clean_images.size(0)

    x0 = clean_images
    t = torch.randint(low=1, high=T+1, size=(batch_size,))

    noise = torch.randn_like(x0, device=x0.device)

    alpha_t = bar_alphas[t-1].view(-1, 1, 1, 1).to(x0.device)

    # *Dodajemy* szum do czystego przykładu
    x_t = alpha_t**0.5 * x0 + (1 - alpha_t)**0.5 * noise

    # Predykcja *dodanego* szumu
    pred_noise = model(x_t, (t / T).view(-1, 1).to(x0.device))

    loss = torch.mean((pred_noise - noise)**2)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()

In [ ]:
model = UNet(time_emb_dim=64).to(get_device())
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = MultiStepLR(optimizer, milestones=[8], gamma=0.1)

num_epochs = 7

for epoch in range(num_epochs):
    total_loss = 0.0
    for clean_images, _ in tqdm(train_loader):
        clean_images = clean_images.to(get_device())
        loss_value = train_step_ddpm(model, clean_images)
        total_loss += loss_value

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}: Loss = {avg_loss:.4f}")
    scheduler.step()

### Próbkowanie

In [ ]:
@torch.no_grad()
def sample_images(model, num_steps: int, num_samples: int, device):
    model.eval()
    # Losowy szum początkowy
    x = torch.randn(num_samples, 1, 28, 28, device=device)

    for step in reversed(range(1, num_steps + 1)):
        t_frac = torch.full((num_samples, 1), step / num_steps, device=device)
        pred_noise = model(x, t_frac)

        # Pobranie skalarów dla bieżącego kroku
        a_bar = bar_alphas[step - 1].view(1, 1, 1, 1)
        a     = alphas[step - 1].view(1, 1, 1, 1)
        b     = betas[step - 1].view(1, 1, 1, 1)

        # Obliczenia mu_t i sigma_t
        mu    = (x - (b / (1 - a_bar).sqrt()) * pred_noise) / a.sqrt()
        sigma = b.sqrt()

        # Dodajemy szum, oprócz ostatniego kroku
        noise = torch.randn_like(x) if step > 1 else 0.
        x = mu + sigma * noise

    return x

# Generujemy i pokazujemy obrazy
samples = sample_images(model, T, num_samples=16, device=device)
show_images(samples.cpu())

# Zadanie

Proszę dokonać porównania modeli generatywnych, tj. VAE, FM/SM oraz DDPM
na zbiorze FashionMNIST lub zbiorze [ludzkich twarzy](https://www.kaggle.com/datasets/badasstechie/celebahq-resized-256x256).
W tym drugim przypadku, obrazy należy przekształcić na skalę szarości
oraz zmienić rozdzielczość na 28x28 pikseli (lub podobną).

Ze względu na "trudniejszy" problem należy rozważyć zwiększenie:
- rozmiaru modelu
- liczby epok treningu.


**(Opcjonalnie)**

Poprawę jakości oraz skrócenie czasu treningu może przynieść przejście z przestrzeni pikseli do przestrzeni
_ukrytej_ za pomocą autokodera, np. 28x28 do 7x7.

W takim przypadku, trening modelu generatywnego dokonuje się dla zakodowanej wersji,
po czym dekoder pozwala powrócić z przestrzeni ukrytej do przestrzeni pikseli.

Ciekawe prace dot. modeli dyfuzyjnych:

- [One Step Diffusion](https://openreview.net/pdf?id=OlzB6LnXcS)
- [NeuralSVG](https://arxiv.org/pdf/2501.03992)
- [Generative emulation of weather forecast ensembles with diffusion models](https://www.science.org/doi/10.1126/sciadv.adk4489)